In [1]:
import pandas as pd
import numpy as np
import regex as re
df = pd.read_csv('ucsd_US_print_dataset_03162023.csv')

In [2]:
len(df)

2169826

# 1. Part 01 of variables
# Serial Number : Remove duplicated rows when all the informations are the same 
1. Some of the rows that have same "serial number" and "MDM ID" but some contains subset of information that other rows contain, so I want to keep only those rows with the most information (have numbers in most columns) and remove the other rows; 
2. Remove all those rows with same "Serial Number" but different "MDM ID"

In [3]:
# Check duplicated serial numbers and the number of duplicated rows 
serial_number_col = 'Serial Number'  # Replace with the actual column name
duplicates = df.groupby(serial_number_col).filter(lambda x: len(x) > 1)
counts = duplicates.groupby(serial_number_col).size().reset_index(name='count').sort_values(by='count', ascending=False)
print(counts)

                           Serial Number  count
100791  ae21aa72c317efa84c7bfacf62c27fed    157
31411   35e7863efe158a5327147311ce2500ab    101
40515   45b4ec1aba9a6ed9a25377079fdf3d7f     77
70418   79624b83b97de51883daddf431257cd3     39
84693   920c3364ecba874f0cad54f28228a5cc     38
...                                  ...    ...
51940   5990db66fd07643a44438675ffde7017      2
51939   5990d07ab763d9d79f1630f8df0db8b1      2
51938   598fcaeb6e815bcda7e03d2d37eb009f      2
51937   598fa8ff50e52ed6a81b49702a6ea6be      2
74279   7ff4f6a18d2e4b64e31d53ca10187515      2

[148558 rows x 2 columns]


In [4]:
# Processing code !!
serial_number_col = 'Serial Number'  # Replace with the actual column name
duplicates = df.groupby(serial_number_col).filter(lambda x: len(x['MDM ID'].unique()) > 1)
df = df.drop_duplicates(subset=[serial_number_col, 'MDM ID'], keep=False)
df = df[~df[serial_number_col].isin(duplicates[serial_number_col].unique())]

In [5]:
# This line can help you check whether there are still duplicated serial numbers in the dataset
if df['Serial Number'].duplicated().any():
    print('There are duplicated values in the column')
else:
    print('There are no duplicated values in the column')

There are no duplicated values in the column


# Printer Total 

In [6]:
df["Printer Total"] = df.groupby("MDM ID")["Serial Number"].transform("nunique")

# Fill Price with Series Median

In [7]:
series = []
for s in df['Product NM']:
    words = s.split()
    first_two_words = ' '.join(words[:2])
    series.append(first_two_words.title())
df["Series"] = series

In [8]:
len(df['Product NM'])

1855736

In [9]:
len(df["Series"])

1855736

In [10]:
avg_price = df.groupby("Series")["Product Price"].median()
print(len(avg_price))
avg_price = avg_price.fillna(avg_price.median())
avg_price

9


Series
Hp Amp           149.99
Hp Deskjet        95.55
Hp Envy          129.99
Hp Ink           149.99
Hp Officejet     217.00
Hp Pagewide      746.74
Hp Photosmart    149.99
Hp Smart         449.99
Hp Tango         115.60
Name: Product Price, dtype: float64

In [11]:
df = pd.merge(df, avg_price, how='left',on='Series',suffixes=['','_avg'])

In [12]:
df[df['Product Price'].isna()][["Product NM", 'Product Price', "Series", "Product Price_avg"]].head(10)

,Product NM,Product Price,Series,Product Price_avg
8,HP OfficeJet 5258 All-in-One Printer,NaN,Hp Officejet,217.00
19,HP OfficeJet 5222 All-in-One Printer,NaN,Hp Officejet,217.00
34,HP ENVY 4501 e-All-in-One Printer,NaN,Hp Envy,129.99
46,HP OfficeJet 5258 All-in-One Printer,NaN,Hp Officejet,217.00
51,HP DeskJet 2732 All-in-One Printer,NaN,Hp Deskjet,95.55
54,HP OfficeJet 5258 All-in-One Printer,NaN,Hp Officejet,217.00
73,HP DeskJet 2732 All-in-One Printer,NaN,Hp Deskjet,95.55
78,HP ENVY 5665 e-All-in-One Printer,NaN,Hp Envy,129.99
83,HP Officejet Pro 8600 e-All-in-One Printer - N...,NaN,Hp Officejet,217.00
86,HP DeskJet Plus 4158 All-in-One Printer,NaN,Hp Deskjet,95.55


In [13]:
df["Product Price"] = df['Product Price'].fillna(df['Product Price_avg'])

In [14]:
df = df.drop(['Product Price_avg'],axis=1)

In [15]:
df[df['Product Price'].isna()][["Product NM", 'Product Price', "Series"]]

,Product NM,Product Price,Series


# HW Revenue

In [16]:
# Group the data by MDM id and sum the price of printers for each user
revenue = df.groupby("MDM ID").agg({"Product Price": "sum"})

df = pd.merge(df, revenue, how='left', on="MDM ID", suffixes=['','_rev'])
df['HW Revenue'] = df['Product Price_rev']
df = df.drop(['Product Price_rev'], axis=1)

In [17]:
df[df['Printer Total'] > 1].head(10).loc[:,['MDM ID', 'Serial Number', 'Printer Total', 'HW Revenue', 'Product Price']]

,MDM ID,Serial Number,Printer Total,HW Revenue,Product Price
6,000765cd6c856e596726a426d1851ff6,3bdecdd9774a3ad293e00c5fef940ca2,2,227.28,131.73
21,0012a94acb15f8d9388e795b9926edc2,895389eaf42b3071949e266963fb9872,2,319.91,170.62
27,001b7b333c8f25b8255effbe363faf56,62f7470a86aa62aa277fb5b213f5b7f5,2,221.13,101.65
76,0033251658e7902d7988b3eba0253e26,2ed0668a49de381e39eeec15878801d8,2,400.61,170.62
82,00351a1728b8f988b3972bae33fcf8c5,61a854849a5c6e73af6a99b2db63baea,2,272.27,101.65
151,007443793a23d96b81f5d8e2cb7d5cbd,3d968cc48e0616ab0348b57764bd4d6e,2,616.99,399.99
178,008696a5b9ca466bc5cf4f6e4cc0f452,05691c7e94f170b96017a212db9cea62,2,512.00,256.00
185,008aed0b7f215fa80bf9ff8fd6cc9c72,eee6c99ddfd041d72ddc1a3158d7d4b0,2,163.31,67.76
186,008be9b5b6e3b76bf8f91afa046a3531,f84aa47b6cff76ec182ea00b44be3edd,2,607.70,210.21
197,009bad18868e23f9d6e7664a9f6f04ad,3edac2a38ecb384330f01137eac54784,2,448.00,256.00


In [18]:
df.columns

Index(['MDM ID', 'Serial Number', 'Printer Total', 'HW Revenue',
       'Registration_Dt', 'First Purchase Date', 'Last Purchase Date',
       'Product ID', 'Product Category NM', 'Product Line NM', 'Part DN',
       'Product NM', 'Product Price', 'Location Desc', 'Age',
       'Household Income', 'Gender', 'Occupation', 'Zip Code',
       'Electronics Computing', 'Household Size', 'Reading General', 'Crafts',
       'Photography', 'Computers', 'Computer Games', 'High Ticket Merchandise',
       'Low Ticket Merchandise', 'Sports', 'Exercise Health', 'Movie Music',
       'Home Improvement', 'High Tech Living', 'Number of Children',
       'Education', 'Purchase Propensity (PP) Laptop',
       'PP Small Format Printer', 'PP General Printer', 'PP Laser Printer',
       'Buying Propensity (BP) Brother Ink', 'BP Canon Ink', 'BP HP Ink',
       'BP Printers Brother', 'BP Priners Canon', 'BP Printers Epson',
       'BP PC Acer', 'BP PC Lenovo', 'IBE2058_AMEX_GOLD',
       'IBE2058_AMEX_REGUL

# First_report last_report

In [19]:
# Convert Registration_Dt column to datetime
df['first_report'] = pd.to_datetime(df['first_report'])
df['last_report'] = pd.to_datetime(df['last_report'])

# New Variable : first report year, last report year

In [20]:
# Extract year, month, and day components
df['first_report_y'] = df['first_report'].dt.year
df['last_report_y'] = df['last_report'].dt.year

In [21]:
df.columns

Index(['MDM ID', 'Serial Number', 'Printer Total', 'HW Revenue',
       'Registration_Dt', 'First Purchase Date', 'Last Purchase Date',
       'Product ID', 'Product Category NM', 'Product Line NM', 'Part DN',
       'Product NM', 'Product Price', 'Location Desc', 'Age',
       'Household Income', 'Gender', 'Occupation', 'Zip Code',
       'Electronics Computing', 'Household Size', 'Reading General', 'Crafts',
       'Photography', 'Computers', 'Computer Games', 'High Ticket Merchandise',
       'Low Ticket Merchandise', 'Sports', 'Exercise Health', 'Movie Music',
       'Home Improvement', 'High Tech Living', 'Number of Children',
       'Education', 'Purchase Propensity (PP) Laptop',
       'PP Small Format Printer', 'PP General Printer', 'PP Laser Printer',
       'Buying Propensity (BP) Brother Ink', 'BP Canon Ink', 'BP HP Ink',
       'BP Printers Brother', 'BP Priners Canon', 'BP Printers Epson',
       'BP PC Acer', 'BP PC Lenovo', 'IBE2058_AMEX_GOLD',
       'IBE2058_AMEX_REGUL

In [22]:
df = df.drop(['Serial Number','Last Purchase Date','First Purchase Date', 'Product ID', 'Product Category NM', 'Product Line NM', 'Part DN','Product NM', 'Location Desc', 'Home Improvement', 'High Tech Living'], axis=1)

In [23]:
df.columns

Index(['MDM ID', 'Printer Total', 'HW Revenue', 'Registration_Dt',
       'Product Price', 'Age', 'Household Income', 'Gender', 'Occupation',
       'Zip Code', 'Electronics Computing', 'Household Size',
       'Reading General', 'Crafts', 'Photography', 'Computers',
       'Computer Games', 'High Ticket Merchandise', 'Low Ticket Merchandise',
       'Sports', 'Exercise Health', 'Movie Music', 'Number of Children',
       'Education', 'Purchase Propensity (PP) Laptop',
       'PP Small Format Printer', 'PP General Printer', 'PP Laser Printer',
       'Buying Propensity (BP) Brother Ink', 'BP Canon Ink', 'BP HP Ink',
       'BP Printers Brother', 'BP Priners Canon', 'BP Printers Epson',
       'BP PC Acer', 'BP PC Lenovo', 'IBE2058_AMEX_GOLD',
       'IBE2058_AMEX_REGULAR', 'IBE2059_DISCOVER_GOLD',
       'IBE2059_DISCOVER_REGULAR', 'IBE2061_MASTERCARD_GOLD',
       'IBE2061_MASTERCARD_REGULAR', 'IBE2062_VISA_GOLD',
       'IBE2062_VISA_REGULAR', 'lifetime_loyalty_class',
       'supply

# 3. Part 03 of Variables -- Print Telemetry & Subscription

## lifetime_loyalty_class

In [24]:
df_copy = df.copy(deep=True)

In [25]:
# Remove NA
df = df.dropna(subset='lifetime_loyalty_class')


## supply_selectability_number

In [26]:
# Return the number of ink types available
df['available_ink_number'] = df['supply_selectability_number'].str.split("/").str.len()
df = df.drop(['supply_selectability_number'], axis = 1)

## user_sub_environment

In [27]:
# Combine "Home" "Office" "Unknown" -> extract the first word
df['user_sub_environment'] = df['user_sub_environment'].str.split().str[0]

## ampv: log + 1

In [28]:
df['ampv_log'] = np.log(df['ampv']+1)

## ave_cc_mo: Remove
## blk_cc_mo, color_cc_mo: log + 1

In [29]:
df = df.drop(['ave_cc_mo'], axis=1)

df['blk_log'] = np.log(df['blk_cc_mo']+1)
df['color_log'] = np.log(df['color_cc_mo']+1)

df = df.drop(['blk_cc_mo', 'color_cc_mo'], axis=1)

## fill na with 1; hp_share = hp_cc/trade_cc

In [30]:
df['hp_share'] = df['hp_share'].fillna(1)

## trade_cc: leave it 
## hp_cc: remove

## days: remove 

In [32]:
df = df.drop(['hp_cc','days'], axis=1)

## subscription_id -> subscribed: replace na with 0, non-na with 1

In [34]:
df['subscribed'] = np.where(df['subscription_id'].isna(), 0, 1)

## subscription_state == "subscribed" -> df['subscription_paid']

In [35]:
df['subscription_paid'] = np.where(df['subscription_state'] == 'subscribed', 1, 0)
df['obsolete'] = np.where((df['subscription_state'] == 'obsolete') | (df['subscription_state'] == 'obsolete_replacement'), 1, 0)
df['unsubscribed'] = np.where((df['subscription_state'] == 'unsubscribed') | (df['subscription_state'] == 'initiated_unsubscribe'), 1, 0)
df = df.drop(['subscription_id', 'enrolled_on_date', 'subscription_date','unsubscribe_date', 'obsolete_date', 'subscription_state'], axis=1)

## current_plan: extract the number part; fillna(0)

In [37]:
df['current_plan'] = df['current_plan'].str.split().str[0].fillna(0)

## Credit Card

In [ ]:
list_mode = ['Age', 'Number of Children', 'Education', 'Household Income', 'Occupation', 'Zip Code', 'region']
for i in list_mode:
    mode_value_temp = df[i].mode()[0]
    df[i].fillna(mode_value_temp, inplace = True)

In [38]:
df = df.reset_index(drop=True)

In [40]:
df_survey_PP = df.dropna(subset = 'PP Laser Printer')
len(df_survey_PP)

115515

In [41]:
for col in df_survey_PP.columns:
    na = sum(df_survey_PP[col].isna())
    if na > 0:
        print(f"{col}:\t{na/len(df_survey_PP)}")

Age:	1.0
Household Income:	0.214708046574038
Gender:	0.11448729602216162
Occupation:	0.5659178461671645
Zip Code:	0.11137081764273038
Electronics Computing:	0.7414448340042419
Household Size:	0.10626325585421807
Reading General:	0.33287451846080596
Crafts:	0.5145738648660347
Photography:	0.8573432021815348
Computers:	0.27532355105397566
Computer Games:	0.9361208501060468
High Ticket Merchandise:	0.10626325585421807
Low Ticket Merchandise:	0.10626325585421807
Sports:	0.443422932086742
Exercise Health:	0.4217807211184695
Movie Music:	0.5243128598017573
Number of Children:	0.35441284681643076
Education:	0.10638445223564039
IBE2058_AMEX_GOLD:	0.10626325585421807
IBE2058_AMEX_REGULAR:	0.10626325585421807
IBE2059_DISCOVER_GOLD:	0.10626325585421807
IBE2059_DISCOVER_REGULAR:	0.10626325585421807
IBE2061_MASTERCARD_GOLD:	0.10626325585421807
IBE2061_MASTERCARD_REGULAR:	0.10626325585421807
IBE2062_VISA_GOLD:	0.10626325585421807
IBE2062_VISA_REGULAR:	0.10626325585421807


In [51]:
df_nonsurvey = df.iloc[list(set(df.index)-set(df_survey_PP.index)),:]

In [52]:
len(df_nonsurvey)

1740186

In [56]:
survey_cols = df.iloc[:,5:36].columns
survey_cols

Index(['Age', 'Household Income', 'Gender', 'Occupation', 'Zip Code',
       'Electronics Computing', 'Household Size', 'Reading General', 'Crafts',
       'Photography', 'Computers', 'Computer Games', 'High Ticket Merchandise',
       'Low Ticket Merchandise', 'Sports', 'Exercise Health', 'Movie Music',
       'Number of Children', 'Education', 'Purchase Propensity (PP) Laptop',
       'PP Small Format Printer', 'PP General Printer', 'PP Laser Printer',
       'Buying Propensity (BP) Brother Ink', 'BP Canon Ink', 'BP HP Ink',
       'BP Printers Brother', 'BP Priners Canon', 'BP Printers Epson',
       'BP PC Acer', 'BP PC Lenovo'],
      dtype='object')

In [57]:
df_nonsurvey = df_nonsurvey.drop(survey_cols, axis=1)

In [58]:
df_nonsurvey.columns

Index(['MDM ID', 'Printer Total', 'HW Revenue', 'Registration_Dt',
       'Product Price', 'lifetime_loyalty_class', 'user_sub_environment',
       'ampv', 'hp_share', 'trade_cc', 'first_report', 'last_report',
       'current_plan', 'Series', 'first_report_y', 'last_report_y',
       'available_ink_number', 'ampv_log', 'blk_log', 'color_log',
       'subscribed', 'subscription_paid', 'obsolete', 'unsubscribed',
       'credit_gold'],
      dtype='object')

In [ ]:
len(df_nonsurvey)

In [ ]:
len(df_survey)

In [ ]:
import os

In [ ]:
os.getcwd()

In [60]:
df_survey_PP.to_csv("C:\\UCD\\Hackathon\\survey_data.csv")

In [59]:
df_nonsurvey.to_csv("C:\\UCD\\Hackathon\\nonsurvey_data.csv")

# 2. Part 02 of Variables -- Demographic & Psychographic Variables

In [ ]:
from uszipcode import SearchEngine
df['Zip Code'] = df['Zip Code'].str.extract(r'^(\d{5})')

#: Replace Zip Codes with corresponding states
zip_code_state_dict = {}

search = SearchEngine()
zip_codes = df['Zip Code'].dropna().unique()

for zip_code in zip_codes:
    result = search.by_zipcode(zip_code)
    if result:
        zip_code_state_dict[zip_code] = result.state

df['Zip Code'] = df['Zip Code'].map(zip_code_state_dict).fillna(df['Zip Code'])

In [ ]:
#: Generate a new column region corresponding to the state
def get_region(state):
    if state in ['ME', 'VT', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'PA']:
        return 'northeast'
    elif state in ['MD', 'DE', 'VA', 'WV', 'KY', 'NC', 'SC', 'TN', 'GA', 'AL', 'MS', 'FL']:
        return 'southeast'
    elif state in ['OH', 'IN', 'IL', 'MI', 'WI', 'MN', 'IA', 'MO', 'ND', 'SD', 'NE', 'KS']:
        return 'midwest'
    elif state in ['TX', 'OK', 'AR', 'LA', 'NM', 'AZ']:
        return 'southwest'
    elif state in ['WA', 'OR', 'CA', 'AK', 'HI', 'MT', 'ID', 'WY', 'NV', 'UT', 'CO']:
        return 'west'
    else:
        return None

df['region'] = df['Zip Code'].apply(get_region)

In [ ]:
# Round Age, # of Children, Education
df[['Age','Number of Children', 'Education']] = df[['Age','Number of Children',"Education"]].apply(lambda x: x.round().astype(int))

## Occupations

In [ ]:
# D-L -> C (Self-employeed); W-Y -> 1 (Professional); AB9V -> Z (Religious, Military, Educator, Farmer -> Other)
df["Occupation"] = df['Occupation'].replace(['[D-L]','[W-Y]','[AB9V]'],["C", '1' ,'Z'],regex=True)

In [49]:
# Combine "Gold" credit card <- only "Gold" has business meaning
df["credit_gold"] = df['IBE2058_AMEX_GOLD'] + df['IBE2061_MASTERCARD_GOLD'] + df['IBE2062_VISA_GOLD']

# Fill na with 0 -> Nongold
df['credit_gold'] = df['credit_gold'].fillna(0)

In [50]:
# Drop credit card related column
df = df.drop(['IBE2058_AMEX_GOLD','IBE2058_AMEX_REGULAR', 'IBE2059_DISCOVER_GOLD','IBE2059_DISCOVER_REGULAR', 'IBE2061_MASTERCARD_GOLD', 'IBE2061_MASTERCARD_REGULAR', 'IBE2062_VISA_GOLD', 'IBE2062_VISA_REGULAR'], axis=1)